4/13/23, 1:48 PM digits-recognizer-simple-xgb-classifier.ipynb -
Colaboratory

> import numpy as np \# linear algebra  
> import pandas as pd \# data processing, CSV file I/O (e.g.
> pd.read_csv)  
> import seaborn as sns  
> import matplotlib.pyplot as plt  
> from sklearn.metrics import accuracy_score  
> from xgboost import XGBClassifier
>
> from sklearn.model_selection import train_test_split  
> from sklearn.preprocessing import LabelEncoder
>
> import os  
> for dirname, \_, filenames in os.walk('/kaggle/input'):  
> for filename in filenames:  
> print(os.path.join(dirname, filename))
>
> /kaggle/input/digit-recognizer/sample_submission.csv  
> /kaggle/input/digit-recognizer/train.csv  
> /kaggle/input/digit-recognizer/test.csv
>
> df_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")  
> df_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
>
> df_train

|           | **label** | **pixel0** | **pixel1** | **pixel2** | **pixel3** | **pixel4** | **pixel5** | **pixel6** | **pixel7** | **pixel8** | **...** | **pixel774** | **pixel775** | **pixel776** | **pixel777** | **pixel77** |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| **0**     | 1         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |
| **1**     | 0         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |
| **2**     | 1         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |
| **3**     | 4         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |
| **4**     | 0         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |
| **...**   | ...       | ...        | ...        | ...        | ...        | ...        | ...        | ...        | ...        | ...        | ...     | ...          | ...          | ...          | ...          |             |
| **41995** | 0         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |
| **41996** | 1         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |
| **41997** | 7         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |
| **41998** | 6         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |
| **41999** | 9         | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | 0          | ...     | 0            | 0            | 0            | 0            |             |

> 42000 rows × 785 columns
>
> <img
> src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image1.png"
> style="width:7.27778in;height:0.15278in" />
>
> df_train.label.unique()
>
> array(\[1, 0, 4, 7, 3, 5, 8, 9, 2, 6\])
>
> <img
> src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image2.png" />Explanatory
> Data Analysis
>
> plt.figure(figsize=(8,6))  
> ax = sns.countplot(x='label',data=df_train)
>
> plt.title("Label Distribution")  
> total= len(df_train.label)  
> for p in ax.patches:  
> percentage = f'{100 \* p.get_height() / total:.1f}%\n'  
> x = p.get_x() + p.get_width() / 2  
> y = p.get_height()  
> ax.annotate(percentage, (x, y), ha='center', va='center')

https://colab.research.google.com/drive/1YvYUELeoV7_6e5D\_\_H6McBVsra0Qnu1W#scrollTo=RiursZUksCTH&printMode=true
1/8

<img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image4.png"
style="width:3.51389in;height:2.6875in" />

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<thead>
<tr class="header">
<th>4/13/23, 1:48 PM</th>
<th><blockquote>
<p>digits-recognizer-simple-xgb-classifier.ipynb - Colaboratory</p>
</blockquote></th>
</tr>
</thead>
<tbody>
</tbody>
</table>

> df_train.describe()

<table style="width:100%;">
<colgroup>
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
<col style="width: 6%" />
</colgroup>
<thead>
<tr class="header">
<th></th>
<th><strong>label</strong></th>
<th><strong>pixel0</strong></th>
<th><strong>pixel1</strong></th>
<th><strong>pixel2</strong></th>
<th><strong>pixel3</strong></th>
<th><strong>pixel4</strong></th>
<th><strong>pixel5</strong></th>
<th><strong>pixel6</strong></th>
<th><strong>pixel7</strong></th>
<th><strong>pixel8</strong></th>
<th><strong>...</strong></th>
<th><strong>pixel774</strong></th>
<th><strong>pixel775</strong></th>
<th><strong>pixe</strong></th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td><strong>count</strong></td>
<td>42000.000000</td>
<td>42000.0</td>
<td>42000.0</td>
<td>42000.0</td>
<td>42000.0</td>
<td>42000.0</td>
<td>42000.0</td>
<td>42000.0</td>
<td>42000.0</td>
<td>42000.0</td>
<td>...</td>
<td>42000.000000</td>
<td>42000.000000</td>
<td><blockquote>
<p>42000.00</p>
</blockquote></td>
</tr>
<tr class="even">
<td><strong>mean</strong></td>
<td>4.456643</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>...</td>
<td>0.219286</td>
<td>0.117095</td>
<td>0.05</td>
</tr>
<tr class="odd">
<td><strong>std</strong></td>
<td>2.887730</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>...</td>
<td>6.312890</td>
<td>4.633819</td>
<td>3.27</td>
</tr>
<tr class="even">
<td><strong>min</strong></td>
<td>0.000000</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>...</td>
<td>0.000000</td>
<td>0.000000</td>
<td>0.00</td>
</tr>
<tr class="odd">
<td><strong>25%</strong></td>
<td>2.000000</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>...</td>
<td>0.000000</td>
<td>0.000000</td>
<td>0.00</td>
</tr>
<tr class="even">
<td><strong>50%</strong></td>
<td>4.000000</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>...</td>
<td>0.000000</td>
<td>0.000000</td>
<td>0.00</td>
</tr>
<tr class="odd">
<td><strong>75%</strong></td>
<td>7.000000</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>...</td>
<td>0.000000</td>
<td>0.000000</td>
<td>0.00</td>
</tr>
<tr class="even">
<td><strong>max</strong></td>
<td>9.000000</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>...</td>
<td><blockquote>
<p>254.000000</p>
</blockquote></td>
<td><blockquote>
<p>254.000000</p>
</blockquote></td>
<td><blockquote>
<p>253.00</p>
</blockquote></td>
</tr>
</tbody>
</table>

> 8 rows × 785 columns
>
> <img
> src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image3.png"
> style="width:7.27778in;height:0.15278in" />
>
> df_train.sum(axis=1)
>
> 0 16650  
> 1 44609  
> 2 13426  
> 3 15029  
> 4 51093  
> ...
>
> 41995 29310  
> 41996 13416  
> 41997 31511  
> 41998 26387  
> 41999 18187  
> Length: 42000, dtype: int64
>
> df_train.shape
>
> (42000, 785)
>
> pixels = df_train.columns.tolist()\[1:\]  
> df_train\["sum"\] = df_train\[pixels\].sum(axis=1)
>
> df_test\["sum"\] = df_test\[pixels\].sum(axis=1)
>
> df_train.groupby(\['label'\])\['sum'\].mean()
>
> label  
> 0 34632.407551  
> 1 15188.466268  
> 2 29871.099354  
> 3 28320.188003  
> 4 24232.722495  
> 5 25835.920422  
> 6 27734.917331  
> 7 22931.244263  
> 8 30184.148413  
> 9 24553.750000  
> Name: sum, dtype: float64

\# separate target values from df train  
https://colab.research.google.com/drive/1YvYUELeoV7_6e5D\_\_H6McBVsra0Qnu1W#scrollTo=RiursZUksCTH&printMode=true
2/8

<table>
<colgroup>
<col style="width: 20%" />
<col style="width: 20%" />
<col style="width: 20%" />
<col style="width: 20%" />
<col style="width: 20%" />
</colgroup>
<thead>
<tr class="header">
<th>4/13/23, 1:48 PM</th>
<th rowspan="2">a ues</th>
<th rowspan="2">o</th>
<th rowspan="2"><blockquote>
<p>d _t a</p>
</blockquote></th>
<th rowspan="2"><blockquote>
<p>digits-recognizer-simple-xgb-classifier.ipynb - Colaboratory</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>sepa ate ta get</th>
</tr>
</thead>
<tbody>
</tbody>
</table>

> targets = df_train.label  
> features = df_train.drop("label",axis=1)
>
> from sklearn.preprocessing import StandardScaler
>
> scaler = StandardScaler()  
> features\[:\] = scaler.fit_transform(features)  
> df_test\[:\] = scaler.transform(df_test)
>
> del df_train
>
> from sklearn.decomposition import PCA as sklearnPCA  
> sklearn_pca = sklearnPCA(n_components=2)  
> Y_sklearn = sklearn_pca.fit_transform(features)
>
> Y_sklearn
>
> array(\[\[-5.27222487, -5.22771721\],  
> \[19.38083905, 6.06277394\],  
> \[-7.83434408, -1.7077624 \],  
> ...,  
> \[ 0.60964676, 7.06775211\],  
> \[ 2.25998582, -4.33641754\],  
> \[-4.89813057, 1.55416705\]\])
>
> #referred to
> https://sebastianraschka.com/Articles/2015_pca_in_3\_steps.html and
> https://www.kaggle.com/arthurtok/interactive-intro-to-dimens
>
> with plt.style.context('seaborn-whitegrid'):  
> plt.figure(figsize=(10, 8))  
> for lab, col in zip((0,1,2,3,4,5,6,7,8,9),  
> ('blue','red','green','yellow','purple','black','brown','pink','orange','beige')):
> plt.scatter(Y_sklearn\[targets==lab, 0\],  
> Y_sklearn\[targets==lab, 1\],  
> label=lab,  
> c=col)  
> plt.xlabel('Principal Component 1')  
> plt.ylabel('Principal Component 2')  
> plt.legend(loc='lower right')  
> plt.tight_layout()  
> plt.show()
>
> <img
> src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image5.png"
> style="width:4.94444in;height:3.94444in" />

features.index  
https://colab.research.google.com/drive/1YvYUELeoV7_6e5D\_\_H6McBVsra0Qnu1W#scrollTo=RiursZUksCTH&printMode=true
3/8

4/13/23, 1:48 PM digits-recognizer-simple-xgb-classifier.ipynb -
Colaboratory

> RangeIndex(start=0, stop=42000, step=1)
>
> sklearn_pca_3 = sklearnPCA(n_components=3)  
> Y_sklearn_3 = sklearn_pca_3.fit_transform(features)  
> Y_sklearn_3\_test = sklearn_pca_3.transform(df_test)
>
> \# Store results of PCA in a data frame  
> result=pd.DataFrame(Y_sklearn_3, columns=\['PCA%i' % i for i in
> range(3)\], index=features.index)
>
> result

<table>
<colgroup>
<col style="width: 25%" />
<col style="width: 25%" />
<col style="width: 25%" />
<col style="width: 25%" />
</colgroup>
<thead>
<tr class="header">
<th></th>
<th><strong>PCA0</strong></th>
<th><strong>PCA1</strong></th>
<th><strong>PCA2</strong></th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td><strong>0</strong></td>
<td>-5.272215</td>
<td>-5.227477</td>
<td><blockquote>
<p>3.888873</p>
</blockquote></td>
</tr>
<tr class="even">
<td><strong>1</strong></td>
<td>19.380779</td>
<td>6.061996</td>
<td><blockquote>
<p>1.339098</p>
</blockquote></td>
</tr>
<tr class="odd">
<td><strong>2</strong></td>
<td>-7.834346</td>
<td>-1.707909</td>
<td><blockquote>
<p>2.291322</p>
</blockquote></td>
</tr>
<tr class="even">
<td><strong>3</strong></td>
<td>-0.706301</td>
<td>5.846836</td>
<td><blockquote>
<p>2.022914</p>
</blockquote></td>
</tr>
<tr class="odd">
<td><strong>4</strong></td>
<td>26.648639</td>
<td>6.067233</td>
<td><blockquote>
<p>0.981754</p>
</blockquote></td>
</tr>
<tr class="even">
<td><strong>...</strong></td>
<td>...</td>
<td>...</td>
<td>...</td>
</tr>
<tr class="odd">
<td><strong>41995</strong></td>
<td>13.527955</td>
<td>-1.320109</td>
<td><blockquote>
<p>-3.914507</p>
</blockquote></td>
</tr>
<tr class="even">
<td><strong>41996</strong></td>
<td>-9.041416</td>
<td>-1.192544</td>
<td><blockquote>
<p>2.320927</p>
</blockquote></td>
</tr>
<tr class="odd">
<td><strong>41997</strong></td>
<td><blockquote>
<p>0.609622</p>
</blockquote></td>
<td>7.067642</td>
<td>-12.100521</td>
</tr>
<tr class="even">
<td><strong>41998</strong></td>
<td><blockquote>
<p>2.259983</p>
</blockquote></td>
<td>-4.336149</td>
<td><blockquote>
<p>0.714092</p>
</blockquote></td>
</tr>
<tr class="odd">
<td><strong>41999</strong></td>
<td>-4.898130</td>
<td>1.554517</td>
<td><blockquote>
<p>-2.500988</p>
</blockquote></td>
</tr>
</tbody>
</table>

> 42000 rows × 3 columns
>
> my_dpi=96  
> plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)
>
> with plt.style.context('seaborn-whitegrid'):  
> my_dpi=96  
> fig = plt.figure(figsize=(10, 10), dpi=my_dpi)  
> ax = fig.add_subplot(111,projection ='3d')  
> for lab, col in zip((0,1,2,3,4,5,6,7,8,9),  
> ('blue','red','green','yellow','purple','black','brown','pink','orange','beige')):
> plt.scatter(Y_sklearn\[targets==lab, 0\],  
> Y_sklearn\[targets==lab, 1\],  
> label=lab,  
> c=col,s =60)
>
> ax.set_xlabel('Principal Component 1')  
> ax.set_ylabel('Principal Component 2')  
> ax.set_zlabel('Principal Component 3')  
> ax.set_title("PCA on the Handwriting Data")  
> plt.show()

| https://colab.research.google.com/drive/1YvYUELeoV7_6e5D\_\_H6McBVsra0Qnu1W#scrollTo=RiursZUksCTH&printMode=true | 4/8 |
|------------------------------------|------------------------------------|

<img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image7.png"
style="width:5.33333in;height:5.29861in" />

4/13/23, 1:48 PM digits-recognizer-simple-xgb-classifier.ipynb -
Colaboratory

> \<Figure size 480x480 with 0 Axes\>
>
> encoder = LabelEncoder()  
> targets\[:\] = encoder.fit_transform(targets\[:\])
>
> X_train,X_val, y_train,y_val =
> train_test_split(result,targets,random_state=1)
>
> <img
> src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image2.png" />Making
> a Model and Predictions
>
> \# 3 Principal Components  
> model = XGBClassifier(max_depth=5, objective='multi:softprob',
> n_estimators=1000, num_classes=10)
>
> history = model.fit(X_train, y_train,eval_set
> =\[(X_val,y_val)\],early_stopping_rounds =50) acc =
> accuracy_score(y_val, model.predict(X_val))  
> print(f"Accuracy: , {round(acc,3)}")

<table>
<colgroup>
<col style="width: 100%" />
</colgroup>
<thead>
<tr class="header">
<th><blockquote>
<p>[5] [6] [7] [8] [9] [10] [11] [12] [13] [14] [15] [16] [17] [18] [19]
[20] [21] validation_0-mlogloss:1.37571 validation_0-mlogloss:1.33634
validation_0-mlogloss:1.30595 validation_0-mlogloss:1.28164
validation_0-mlogloss:1.26131 validation_0-mlogloss:1.24565
validation_0-mlogloss:1.23126 validation_0-mlogloss:1.21949
validation_0-mlogloss:1.20960 validation_0-mlogloss:1.20136
validation_0-mlogloss:1.19384 validation_0-mlogloss:1.18698
validation_0-mlogloss:1.18185 validation_0-mlogloss:1.17790
validation_0-mlogloss:1.17392 validation_0-mlogloss:1.17091
validation_0-mlogloss:1.16766 <img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image6.png"
style="width:0.15278in;height:4.04167in" /></p>
<p>/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224:
UserWarning: The use of label encoder in XGBClassifier is deprecated
warnings.warn(label_encoder_deprecation_msg, UserWarning)<br />
[06:57:47] WARNING: ../src/learner.cc:576:<br />
Parameters: { "num_classes" } might not be used.</p>
<p>This could be a false alarm, with some parameters getting used by
language bindings but then being mistakenly passed down to XGBoost core,
or some parameter actually being used but getting flagged wrongly here.
Please open an issue if you find any such cases.</p>
<p>[06:57:47] WARNING: ../src/learner.cc:1115: Starting in XGBoost
1.3.0, the default evaluation metric used with the objective 'multi:s
[0] validation_0-mlogloss:1.87607<br />
[1] validation_0-mlogloss:1.69509<br />
[2] validation_0-mlogloss:1.57431<br />
[3]<br />
validation_0-mlogloss:1.49011 [4] validation_0-mlogloss:1.42468</p>
</blockquote></th>
</tr>
</thead>
<tbody>
</tbody>
</table>

https://colab.research.google.com/drive/1YvYUELeoV7_6e5D\_\_H6McBVsra0Qnu1W#scrollTo=RiursZUksCTH&printMode=true
5/8

<img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image9.png"
style="width:7.27778in;height:2.97222in" />

<table>
<colgroup>
<col style="width: 33%" />
<col style="width: 33%" />
<col style="width: 33%" />
</colgroup>
<thead>
<tr class="header">
<th>4/13/23, 1:48 PM</th>
<th rowspan="2"><blockquote>
<p>validation_0-mlogloss:1.16477</p>
</blockquote></th>
<th rowspan="24"><blockquote>
<p>digits-recognizer-simple-xgb-classifier.ipynb - Colaboratory</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[22]</th>
</tr>
<tr class="header">
<th>[23]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.16259</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[24]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.16109</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[25]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15964</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[26]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15824</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[27]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15685</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[28]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15600</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[29]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15523</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[30]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15382</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[31]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15289</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[32]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15221</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[33]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15184</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[34]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15136</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[35]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15087</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[36]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15040</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[37]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.15031</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[38]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.14978</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[39]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.14911</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[40]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.14890</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[41]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.14878</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[42]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.14867</p>
</blockquote></th>
</tr>
<tr class="header">
<th>[43]</th>
<th><blockquote>
<p>validation_0-mlogloss:1.14846</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[44]</th>
<th><blockquote>
<p>validation 0-mlogloss:1.14829</p>
</blockquote></th>
</tr>
</thead>
<tbody>
</tbody>
</table>

> X_train,X_val, y_train,y_val =
> train_test_split(features,targets,random_state=1)
>
> model = XGBClassifier(max_depth=5, objective='multi:softprob',
> n_estimators=1000, num_classes=10)
>
> history = model.fit(X_train, y_train,eval_set
> =\[(X_train,y_train),(X_val,y_val)\],early_stopping_rounds =5) acc =
> accuracy_score(y_val, model.predict(X_val))  
> print(f"Accuracy: , {round(acc,3)}")
>
> \[14\] \[15\] \[16\] \[17\] \[18\] \[19\] \[20\] \[21\] \[22\] \[23\]
> \[24\] \[25\] \[26\] \[27\] \[28\] \[29\] \[30\] \[31\] \[32\] \[33\]
> \[34\] \[35\] \[36\] validation_0-mlogloss:0.18458
> validation_0-mlogloss:0.16888 validation_0-mlogloss:0.15593
> validation_0-mlogloss:0.14441 validation_0-mlogloss:0.13361
> validation_0-mlogloss:0.12413 validation_0-mlogloss:0.11625
> validation_0-mlogloss:0.10815 validation_0-mlogloss:0.10031
> validation_0-mlogloss:0.09473 validation_0-mlogloss:0.08789
> validation_0-mlogloss:0.08249 validation_0-mlogloss:0.07738
> validation_0-mlogloss:0.07189 validation_0-mlogloss:0.06801
> validation_0-mlogloss:0.06414 validation_0-mlogloss:0.06057
> validation_0-mlogloss:0.05713 validation_0-mlogloss:0.05349
> validation_0-mlogloss:0.05094 validation_0-mlogloss:0.04831
> validation_0-mlogloss:0.04560 validation_0-mlogloss:0.04306
> validation_1-mlogloss:0.25708 validation_1-mlogloss:0.24343
> validation_1-mlogloss:0.23174 validation_1-mlogloss:0.22166
> validation_1-mlogloss:0.21180 validation_1-mlogloss:0.20371
> validation_1-mlogloss:0.19627 validation_1-mlogloss:0.18900
> validation_1-mlogloss:0.18205 validation_1-mlogloss:0.17735
> validation_1-mlogloss:0.17083 validation_1-mlogloss:0.16605
> validation_1-mlogloss:0.16129 validation_1-mlogloss:0.15697
> validation_1-mlogloss:0.15350 validation_1-mlogloss:0.14999
> validation_1-mlogloss:0.14722 validation_1-mlogloss:0.14373
> validation_1-mlogloss:0.14027 validation_1-mlogloss:0.13833
> validation_1-mlogloss:0.13593 validation_1-mlogloss:0.13382
> validation_1-mlogloss:0.13161 <img
> src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image8.png"
> style="width:0.15278in;height:5.70833in" />
>
> \[06:58:04\] WARNING: ../src/learner.cc:576:  
> Parameters: { "num_classes" } might not be used.
>
> This could be a false alarm, with some parameters getting used by
> language bindings but then being mistakenly passed down to XGBoost
> core, or some parameter actually being used but getting flagged
> wrongly here. Please open an issue if you find any such cases.
>
> \[06:58:08\] WARNING: ../src/learner.cc:1115: Starting in XGBoost
> 1.3.0, the default evaluation metric used with the objective 'multi:s
> \[0\] validation_0-mlogloss:1.42839 validation_1-mlogloss:1.44561  
> \[1\] validation_0-mlogloss:1.09931 validation_1-mlogloss:1.13044  
> \[2\]  
> validation_0-mlogloss:0.88151 validation_1-mlogloss:0.92129 \[3\]
> validation_0-mlogloss:0.72542 validation_1-mlogloss:0.77069  
> \[4\] validation_0-mlogloss:0.60957 validation_1-mlogloss:0.65913  
> \[5\] validation_0-mlogloss:0.51863 validation_1-mlogloss:0.57078  
> \[6\] validation_0-mlogloss:0.44818 validation_1-mlogloss:0.50382  
> \[7\] validation_0-mlogloss:0.38978  
> validation_1-mlogloss:0.44809 \[8\] validation_0-mlogloss:0.34231
> validation_1-mlogloss:0.40327  
> \[9\] validation_0-mlogloss:0.30416 validation_1-mlogloss:0.36713  
> \[10\]  
> validation_0-mlogloss:0.27249 validation_1-mlogloss:0.33768 \[11\]
> validation_0-mlogloss:0.24479 validation_1-mlogloss:0.31212  
> \[12\] validation_0-mlogloss:0.22142 validation_1-mlogloss:0.29098  
> \[13\] validation_0-mlogloss:0.20190 validation_1-mlogloss:0.27290

https://colab.research.google.com/drive/1YvYUELeoV7_6e5D\_\_H6McBVsra0Qnu1W#scrollTo=RiursZUksCTH&printMode=true
6/8

<img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image10.png"
style="width:7.27778in;height:1.30556in" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image11.png"
style="width:4.22917in;height:3.33333in" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image12.png" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image12.png" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image13.png"
style="width:0.40278in;height:0.23611in" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image14.png"
style="width:3.55556in;height:2.79167in" />

<table>
<colgroup>
<col style="width: 33%" />
<col style="width: 33%" />
<col style="width: 33%" />
</colgroup>
<thead>
<tr class="header">
<th>4/13/23, 1:48 PM</th>
<th rowspan="2">validation_0-mlogloss:0.04033</th>
<th><blockquote>
<p>digits-recognizer-simple-xgb-classifier.ipynb - Colaboratory</p>
</blockquote></th>
</tr>
<tr class="odd">
<th>[37]</th>
<th><blockquote>
<p>validation_1-mlogloss:0.12903</p>
</blockquote></th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td>[38]</td>
<td>validation_0-mlogloss:0.03796</td>
<td><blockquote>
<p>validation_1-mlogloss:0.12721</p>
</blockquote></td>
</tr>
<tr class="even">
<td>[180]</td>
<td>validation_0-mlogloss:0.00084</td>
<td><blockquote>
<p>validation_1-mlogloss:0.08112</p>
</blockquote></td>
</tr>
<tr class="odd">
<td>[181]</td>
<td>validation_0-mlogloss:0.00083</td>
<td><blockquote>
<p>validation_1-mlogloss:0.08109</p>
</blockquote></td>
</tr>
<tr class="even">
<td>[182]</td>
<td>validation_0-mlogloss:0.00082</td>
<td><blockquote>
<p>validation_1-mlogloss:0.08103</p>
</blockquote></td>
</tr>
<tr class="odd">
<td>[183]</td>
<td>validation_0-mlogloss:0.00081</td>
<td><blockquote>
<p>validation_1-mlogloss:0.08103</p>
</blockquote></td>
</tr>
<tr class="even">
<td>[184]</td>
<td>validation_0-mlogloss:0.00081</td>
<td><blockquote>
<p>validation_1-mlogloss:0.08102</p>
</blockquote></td>
</tr>
<tr class="odd">
<td>[185]</td>
<td>validation_0-mlogloss:0.00080</td>
<td><blockquote>
<p>validation_1-mlogloss:0.08100</p>
</blockquote></td>
</tr>
<tr class="even">
<td>[186]</td>
<td>validation_0-mlogloss:0.00080</td>
<td><blockquote>
<p>validation_1-mlogloss:0.08102</p>
</blockquote></td>
</tr>
</tbody>
</table>

> results = model.evals_result()
>
> from matplotlib import pyplot  
> \# plot learning curves  
> plt.figure(figsize=(10, 8))  
> pyplot.plot(results\['validation_0'\]\['mlogloss'\], label='train')  
> pyplot.plot(results\['validation_1'\]\['mlogloss'\], label='test')  
> \# show the legend  
> pyplot.legend()  
> plt.xlabel('iterations')  
> plt.ylabel('mlogloss')  
> \# show the plot  
> pyplot.show()
>
> from xgboost import plot_importance  
> ax = plot_importance(model,max_num_features=10)  
> fig = ax.figure  
> fig.set_size_inches(10,8)  
> plt.show()

https://colab.research.google.com/drive/1YvYUELeoV7_6e5D\_\_H6McBVsra0Qnu1W#scrollTo=RiursZUksCTH&printMode=true
7/8

<img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image16.png"
style="width:4.41667in;height:3.44444in" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image17.png" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image18.png" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image19.png" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image20.png" /><img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image21.png" />

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<thead>
<tr class="header">
<th>4/13/23, 1:48 PM</th>
<th><blockquote>
<p>digits-recognizer-simple-xgb-classifier.ipynb - Colaboratory</p>
</blockquote></th>
</tr>
</thead>
<tbody>
</tbody>
</table>

> predictions = model.predict(df_test)
>
> output =
> pd.read_csv("../input/digit-recognizer/sample_submission.csv")  
> output\['Label'\] = predictions  
> output.to_csv('submission.csv',index=False)

\-

<img
src="attachment:vertopal_d7ff2b579dd74c0ea27b50b1917457dc/media/image15.png"
style="width:0.25in" />

https://colab.research.google.com/drive/1YvYUELeoV7_6e5D\_\_H6McBVsra0Qnu1W#scrollTo=RiursZUksCTH&printMode=true
8/8